<a href="https://colab.research.google.com/github/giorgiaceccarelli/giorgiaceccarelli.github.io/blob/main/Codice_tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install transformers tensorflow

In [16]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Get the file name from the uploaded files
uploaded_file_name = list(uploaded.keys())[0]

# Read the file with the correct delimiter
unlabelled_tweets_df = pd.read_csv(uploaded_file_name, sep=';', encoding='utf-8')

# Display the first few rows of the dataset to understand its structure and contents
unlabelled_tweets_df.head()

Saving tweets_remaining_09042020_16072020.csv to tweets_remaining_09042020_16072020 (1).csv


,id,created_at,full_text
0,1,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...
1,2,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...
2,3,2020-04-09 23:58:52+00:00,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,2020-04-09 23:58:27+00:00,@Issaquahfunds Hedged our $MSFT position into ...
4,5,2020-04-09 23:57:59+00:00,RT @zipillinois: 3 Surprisingly Controversial ...


In [23]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import re

# Upload the file
uploaded = files.upload()

# Get the file name from the uploaded files
uploaded_file_name = list(uploaded.keys())[0]

# Read the file with the correct delimiter
labelled_tweets_df = pd.read_csv(uploaded_file_name, sep=';', encoding='utf-8')

# Remove rows with NaN values in the sentiment column
labelled_tweets_df = labelled_tweets_df.dropna(subset=['sentiment'])

# Display the first few rows of the dataset to understand its structure and contents
print(labelled_tweets_df.head())

# Display unique sentiment values
unique_sentiments = labelled_tweets_df['sentiment'].unique()
print(unique_sentiments)

# Preprocess text data
def preprocess_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = text.lower()
    return text

labelled_tweets_df['text'] = labelled_tweets_df['text'].apply(preprocess_text)

# Encode sentiment labels
label_encoder = LabelEncoder()
labelled_tweets_df['sentiment'] = label_encoder.fit_transform(labelled_tweets_df['sentiment'].astype(str))

# Check encoded sentiment values to ensure they are in the correct range
print(labelled_tweets_df['sentiment'].unique())

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(labelled_tweets_df['text'], labelled_tweets_df['sentiment'], test_size=0.2, random_state=42)

# Use BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences in the training and testing sets
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).shuffle(100).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Ensure labels are of type 'int32'
train_dataset = train_dataset.map(lambda x, y: (x, tf.cast(y, tf.int32)))
test_dataset = test_dataset.map(lambda x, y: (x, tf.cast(y, tf.int32)))

# Load the BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
error_rate = 1 - accuracy
print(f'Error Rate: {error_rate}')


Saving tweets_labelled_09042020_16072020.csv to tweets_labelled_09042020_16072020 (8).csv
       id                 created_at  \
0   77522  2020-04-15 01:03:46+00:00   
1  661634  2020-06-25 06:20:06+00:00   
2  413231  2020-06-04 15:41:45+00:00   
3  760262  2020-07-03 19:39:35+00:00   
4  830153  2020-07-09 14:39:14+00:00   

                                                text sentiment  
0  RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...  positive  
1  #SriLanka surcharge on fuel removed!\n⛽📉\nThe ...  negative  
2  Net issuance increases to fund fiscal programs...  positive  
3  RT @bentboolean: How much of Amazon's traffic ...  positive  
4  $AMD Ryzen 4000 desktop CPUs looking ‘great’ a...  positive  
['positive' 'negative' 'neutral']
[2 0 1]


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
33/33 [==============================] - 1366s 40s/step - loss: 1.0608 - accuracy: 0.4452 - val_loss: 0.9906 - val_accuracy: 0.4923
Epoch 2/3
33/33 [==============================] - 1295s 39s/step - loss: 0.8222 - accuracy: 0.6404 - val_loss: 0.8177 - val_accuracy: 0.6538
Epoch 3/3
9/9 [==============================] - 69s 8s/step - loss: 0.9378 - accuracy: 0.6692
Error Rate: 0.33076924085617065
